In [1]:
import keras
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Input, BatchNormalization, AveragePooling2D, Reshape, Activation
from keras.layers.advanced_activations import PReLU
from keras import backend as K
from math import sqrt
import sys
from TB_writer import TB_writer
sys.path.append('../CustomLayers/')
from CustomLayers import *

Using TensorFlow backend.


In [4]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
        '/data/cifar/train/',
        target_size=(32,32),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/data/cifar/test/',
        target_size=(32,32),
        batch_size=32,
        class_mode='categorical')

Found 50000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [6]:
img_w = 32
img_h = 32
img_c = 3
inp = Input(shape=(img_w, img_h, img_c))

z = Convolution2D(32, (3,3), activation='relu')(inp)
z = MaxPooling2D(pool_size=(3,3), strides=(2,2))(z)

z = BatchNormalization()(z)
#z = Convolution2D(32, (3,3), activation='relu')(z)
#z = BinLayer()(z)
z = MultibitLayer(3)(z)
z = BinConv(128, (3,3), kernel_regularizer=BinReg(), padding='same')(z)
z = PReLU()(z)
z = MaxPooling2D(pool_size=(3,3), strides=(2,2))(z)

z = BatchNormalization()(z)
#z = BinLayer()(z)
z = MultibitLayer(3)(z)
z = BinConv(128, (3,3), kernel_regularizer=BinReg(), padding='same')(z)
z = PReLU()(z)
z = MaxPooling2D(pool_size=(3,3), strides=(2,2))(z)

z = BatchNormalization()(z)
z = Convolution2D(10, (1,1), activation='relu')(z)
z = AveragePooling2D(pool_size=(int(z.shape[1]), int(z.shape[2])))(z)
z = Reshape((10,))(z)
z = Activation('softmax')(z)

model = Model(inputs=inp, outputs=z)

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
multibit_layer_1 (MultibitLa (None, 14, 14, 32)        0         
_________________________________________________________________
bin_conv_1 (BinConv)         (None, 14, 14, 128)       36992     
_________________________________________________________________
p_re_lu_1 (PReLU)            (None, 14, 14, 128)       25088     
__________

In [9]:
tb_callback = TB_writer(histogram_freq=1, write_images=True, log_dir="cifar_test_binary", val_gen=validation_generator)
tb_callback.set_model(model)

In [10]:
model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=100,
        callbacks=[tb_callback])

Epoch 1/10
100/100 [==============================] - 3s - loss: 2.5142 - acc: 0.2972 - val_loss: 3.3439 - val_acc: 0.1862
Epoch 2/10
100/100 [==============================] - 2s - loss: 2.2553 - acc: 0.3991 - val_loss: 4.6993 - val_acc: 0.1775
Epoch 3/10
100/100 [==============================] - 2s - loss: 2.1618 - acc: 0.4309 - val_loss: 3.9613 - val_acc: 0.1609
Epoch 4/10
100/100 [==============================] - 2s - loss: 2.0620 - acc: 0.4731 - val_loss: 3.6179 - val_acc: 0.1643
Epoch 5/10
100/100 [==============================] - 2s - loss: 2.0338 - acc: 0.4756 - val_loss: 2.4319 - val_acc: 0.3647
Epoch 6/10
100/100 [==============================] - 2s - loss: 2.0175 - acc: 0.4869 - val_loss: 2.0824 - val_acc: 0.4500
Epoch 7/10
100/100 [==============================] - 2s - loss: 1.9792 - acc: 0.4934 - val_loss: 2.0066 - val_acc: 0.5006
Epoch 8/10
100/100 [==============================] - 2s - loss: 1.9498 - acc: 0.5147 - val_loss: 1.9076 - val_acc: 0.5066
Epoch 9/10
100/1

In [11]:
from PIL import Image

In [12]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="float32" )
    data = data/255
    return data

In [13]:
image = load_image("/data/cifar/test/truck/1008_truck.png")
image = image.reshape((1,)+image.shape)

In [14]:
class_map =validation_generator.class_indices

In [15]:
guess = np.argmax(model.predict(image))

In [16]:
validation_generator.class_indices

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [17]:
cifar_labels=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [18]:
cifar_labels[guess]

'automobile'